In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model

function to create label

In [ ]:
# function to create label(dictionary)
def load_datasets(dataset_categories):
    images = []
    labels = []

    for dataset_id, (dataset_path, category) in enumerate(dataset_categories.items()):
        for object_category in os.listdir(dataset_path):
            category_path = os.path.join(dataset_path, object_category)

            for file in os.listdir(category_path):
                img_path = os.path.join(category_path, file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))  # resize all images
                images.append(img)
                labels.append({
                    'category': category,
                    'dataset_id': dataset_id
                })

    return images, labels

loading datasets

In [ ]:
# define dataset paths and corresponding category names
dataset_categories = {
    r'images\bee_image_sample': 'bee',
    r'images\football_image_sample': 'football',
    r'images\keyboard_image_sample': 'keyboard',
    r'images\laptop_image_sample': 'laptop',
    r'images\Letter_M_image_sample': 'letter M',
    r'images\Letter_T_image_sample': 'letter T',
    r'images\monitor_image_sample': 'monitor',
    r'images\mouse_image_sample': 'mouse',
    r'images\trains_image_sample': 'trains',
}

In [ ]:
# verifying dataset is loaded correctly 
for dataset_path, category in dataset_categories.items():
    num_images = sum(len(files) for _, _, files in os.walk(dataset_path))
    print(f"dataset: {dataset_path}, category: {category}, no. of images: {num_images}")

load and preprocess the datasets

In [ ]:
images, labels = load_datasets(dataset_categories)
print(images,labels)

# convert labels to numerical format
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform([label['category'] for label in labels])
encoded_labels = to_categorical(encoded_labels)

saving LabelEncoder

In [ ]:
# convert labels to numerical format
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform([label['category'] for label in labels])
encoded_labels = to_categorical(encoded_labels)

# save the label encoder to a file
np.save('label_encoder_classes.npy', label_encoder.classes_)

splitting into train / test data

In [ ]:
# split the dataset into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(np.array(images), encoded_labels, test_size=0.2, random_state=42)

print("Number of Datasets:", len(dataset_categories))

building the CNN model

In [ ]:

num_classes = 9
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training the model

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

evaluate the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

make predictions

In [ ]:
predictions = model.predict(X_test)

save and load model

In [ ]:
model.save('object_recognition_model.h5')  # save model

# load model
loaded_model = load_model('object_recognition_model.h5')